In [ ]:
import os
import glob
import optuna
import warnings
import h5py as h5
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from copy import deepcopy

from gensit.config import Config
from gensit.inputs import Inputs
from gensit.outputs import Outputs
from gensit.utils.misc_utils import *
from gensit.utils.math_utils import *
from gensit.utils.probability_utils import *
from gensit.contingency_table import instantiate_ct
from gensit.contingency_table.MarkovBasis import instantiate_markov_basis

In [ ]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

## Import samples

In [ ]:
# Specify experiment id
experiment_id = "JointTableSIM_NN_SweepedNoise_01_02_2024_14_42_49"
# Specify experiment group id
experiment_group_id = 'exp2/'
experiment_dir = f'../data/outputs/cambridge/{experiment_group_id}/{experiment_id}/'
relative_experiment_dir = os.path.relpath(experiment_dir,os.getcwd())

In [ ]:
# Create new logging object
logger = setup_logger(
    __name__,
    console_level = 'INFO',
    file_level = 'EMPTY'
)

In [ ]:
# Output processing settings
settings = {
    "logging_mode": "INFO",
    "coordinate_slice": [
        "da.loss_name.isin([str(['dest_attraction_ts_likelihood_loss']),str(['dest_attraction_ts_likelihood_loss', 'table_likelihood_loss'])])",
        "~da.title.isin(['_unconstrained','_total_intensity_row_table_constrained'])",
        # "da.loss_name == str(['dest_attraction_ts_likelihood_loss'])",
        # "~da.title.isin([str('_unconstrained'), str('_total_constrained')])"
    ],
    "slice":True,
    "metadata_keys":[],
    "burnin_thinning_trimming": [], # {'iter': {"burnin":10000, "thinning":90, "trimming":1000}}
    "sample":["table"],
    "group_by":['seed'],
    "filename_ending":"test",
    "force_reload":False,
    "n_workers": 1
}

In [ ]:
# Initialise outputs
current_sweep_outputs = Outputs(
    config = os.path.join(relative_experiment_dir,"config.json"),
    settings = settings,
    inputs = None,
    slice = True,
    level = 'NOTE'
)
# Silence outputs
current_sweep_outputs.logger.setLevels(console_level='NOTE')
# Load all data
current_sweep_outputs.load()

In [ ]:
# Get data from first sweep of the SIM_NN experiment
current_sweep_outputs0 = current_sweep_outputs.get(0)

In [ ]:
# Get table samples
current_sweep_outputs0.data.table